In [2]:
import pandas as pd
from web3 import Web3
import yaml


# Connecting RPCs

In [3]:
# Ethereum mainnet - using a free api rpc like alchemy or infura

with open(r'../eth_rpc.yaml') as file:
    provider_params = yaml.load(file, Loader=yaml.FullLoader)
provider_url = provider_params['key']
w3eth = Web3(Web3.HTTPProvider(provider_url))

w3eth.isConnected()

True

In [4]:
# gnosischain mainnet

xdai_rpc_link = 'https://rpc.gnosischain.com/'

w3x = Web3(Web3.HTTPProvider(xdai_rpc_link))

w3x.isConnected()

True

# Running test queries and testing stuff

### Testing: How do the tranfers from gc to ethereum work?

In [9]:
poap_address = "0x22C1f6050E56d2876009903609a2cC3fEf83B415"
with open('poap_abi.json', 'r') as file:
    abi = file.read()

# POAP contract on ethereum
poap_eth = w3eth.eth.contract(address=poap_address, abi=abi)
eth_total_supply = poap_eth.functions.totalSupply().call()

# POAP contract on gc
poap_gc = w3x.eth.contract(address=poap_address, abi=abi)
gc_total_supply = poap_gc.functions.totalSupply().call()

print(f" Supplies for both contracts are equal? A: {eth_total_supply == gc_total_supply} \n Total supply on ethereum: {eth_total_supply} \n Total supply on gc: {gc_total_supply}")


 Supplies for both contracts are equal? A: False 
 Total supply on ethereum: 34506 
 Total supply on gc: 3651609


In [10]:

#Example where the token is only on ethereum contract // gc URI is invalid

token_id = 9986

uri_eth = poap_eth.functions.tokenURI(token_id).call()

uri_gc = poap_gc.functions.tokenURI(token_id).call()

print(f"ethereum URI: {uri_eth} \n  gc URI: {uri_gc}")


ethereum URI: https://api.poap.xyz/metadata/207/9986 
  gc URI: https://api.poap.xyz/metadata/0/9986


In [11]:

#Example where the token is only on ethereum contract // eth URI is invalid

token_id = 300000

uri_eth = poap_eth.functions.tokenURI(token_id).call()

uri_gc = poap_gc.functions.tokenURI(token_id).call()

print(f"ethereum URI: {uri_eth} \n  gc URI: {uri_gc}")


ethereum URI: https://api.poap.xyz/metadata/0/300000 
  gc URI: https://api.poap.xyz/metadata/2598/300000


### Testing: Can I get the event minters from the event EventMinterAdded?

Testing a solution that doesn't require an archival or full node to get the data. 

Credit to Mikko Ohtamaa on : https://ethereum.stackexchange.com/questions/51637/get-all-the-past-events-of-the-contract


In [12]:
import requests

from web3 import Web3

_cache = dict()


def get_cached_abi(abi_url):
    """Per process over-the-network ABI file retriever"""
    spec = _cache.get(abi_url)
    if not spec:
      spec = _cache[abi_url] = requests.get(abi_url).json()
    return spec


def create_contract(web3, abi_path, address):
    with open(abi_path, 'r') as file:
      abi = file.read()
    contract = web3.eth.contract(address, abi=abi)
    return contract

In [13]:
from web3._utils.abi import get_constructor_abi, merge_args_and_kwargs
from web3._utils.events import get_event_data
from web3._utils.filters import construct_event_filter_params
from web3._utils.contracts import encode_abi


def fetch_events(
    event,
    argument_filters=None,
    from_block=None,
    to_block="latest",
    address=None,
    topics=None):
    """Get events using eth_getLogs API.

    This is a stateless method, as opposite to createFilter and works with
    stateless nodes like QuikNode and Infura.

    :param event: Event instance from your contract.events
    :param argument_filters:
    :param from_block: Start block. Use 0 for all history/
    :param to_block: Fetch events until this contract
    :param address:
    :param topics:
    :return:
    """

    if from_block is None:
        raise TypeError("Missing mandatory keyword argument to getLogs: from_Block")

    abi = event._get_event_abi()
    abi_codec = event.web3.codec

    # Set up any indexed event filters if needed
    argument_filters = dict()
    _filters = dict(**argument_filters)

    data_filter_set, event_filter_params = construct_event_filter_params(
        abi,
        abi_codec,
        contract_address=event.address,
        argument_filters=_filters,
        fromBlock=from_block,
        toBlock=to_block,
        address=address,
        topics=topics,
    )

    # Call node over JSON-RPC API
    logs = event.web3.eth.getLogs(event_filter_params)

    # Convert raw binary event data to easily manipulable Python objects
    for entry in logs:
        data = get_event_data(abi_codec, abi, entry)
        yield data

In [14]:
# from web3 import Web3
# from .utils import create_contract
# from .events import fetch_events


# uniswap_factory = '0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f'
# factory_abi_url = 'https://unpkg.com/@uniswap/v2-core@1.0.1/build/UniswapV2Factory.json'
# erc20_abi_url = 'https://unpkg.com/@uniswap/v2-core@1.0.1/build/IERC20.json'

def fetch_uniswap_pairs(web3: Web3):
    """Fetch all trading pairs on Uniswap"""
    contract = create_contract(web3, './poap_abi.json', poap_address)
    events = list(fetch_events(contract.events.EventMinterAdded, from_block=0))

    print('Got', len(events), 'events')

    # Each event.args is presented as AttrbuteDict
    # AttributeDict({'args': AttributeDict({'token0': '0x607F4C5BB672230e8672085532f7e901544a7375', 'token1': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', 'pair': '0x6d57a53A45343187905aaD6AD8eD532D105697c1', '': 94}), 'event': 'PairCreated', 'logIndex': 7, 'transactionIndex': 2, 'transactionHash': HexBytes('0xa0ce4b0db9bbf7887f09c4b35ec1167144b06f69fbbea6d6a163a72db28175d8'), 'address': '0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f', 'blockHash': HexBytes('0xf269a89cf729781bfa8e8ec421f8eefbf13e1fecd22b4118c1304d360832ef20'), 'blockNumber': 10092190})
    for ev in events[0:10]:
        print(ev)
    #   token0 = create_contract(web3, erc20_abi_url, ev.args.token0)
    #   token1 = create_contract(web3, erc20_abi_url, ev.args.token1)
    #   print(f'Found pair {token0.functions.symbol().call()}-{token1.functions.symbol().call()}')


def run(web3):
    # infura = # Your HTTPS infura endpoint URL
    # web3 = Web3(Web3.HTTPProvider(infura))
    fetch_uniswap_pairs(web3)


run(w3x)


Got 32 events
AttributeDict({'args': AttributeDict({'eventId': 421, 'account': '0xDBEa2EE858158f1c2a0000cdAA0Ab1eD641A4944'}), 'event': 'EventMinterAdded', 'logIndex': 2, 'transactionIndex': 2, 'transactionHash': HexBytes('0x042422bb368c0cbb7c9cd2e650acf4e0d331520f44f9aaadff4c360f43fcf99f'), 'address': '0x22C1f6050E56d2876009903609a2cC3fEf83B415', 'blockHash': HexBytes('0x4d21e9263469a4c9acfc649f6edbc3fbf4c76c0ed56816182c7bdf105133fe43'), 'blockNumber': 12294643})
AttributeDict({'args': AttributeDict({'eventId': 416, 'account': '0x32A3c33C5324838077913AeFF595533c2bBdf561'}), 'event': 'EventMinterAdded', 'logIndex': 4, 'transactionIndex': 3, 'transactionHash': HexBytes('0x1d29d7e0d6cf1bb8dfa28118d36b38e7503b10b45d757c925ff01ff9ccfff945'), 'address': '0x22C1f6050E56d2876009903609a2cC3fEf83B415', 'blockHash': HexBytes('0x5fba762f2be092c19c726cfe4677f5650fa2d033129044a3b6e00fb84ebacd49'), 'blockNumber': 12533702})
AttributeDict({'args': AttributeDict({'eventId': 418, 'account': '0x32A3c33C

The above works! I can use that to generate a new dataframe that I can merge with the token data dataframe!

In [20]:
events_dt = pd.read_json('../analysis/datasets/event_data.json')
len(events_dt)

24219

In [21]:

ethp = pd.read_json('../analysis/datasets/ethereum_token_data.json')
len(ethp)

34741

In [22]:

xdai = pd.read_json('../analysis/datasets/xdai_token_data.json')
len(xdai)

3670459

In [24]:
xdai.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3670459 entries, 0 to 3670458
Data columns (total 8 columns):
token_created          int64
event_created          int64
event_id               int64
event_tokenCount       int64
event_transferCount    int64
token_id               int64
owner_id               object
token_transferCount    int64
dtypes: int64(7), object(1)
memory usage: 224.0+ MB
